In [5]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

if TMDB_API_KEY and TMDB_API_TOKEN:
    print('✅API KEY and TOKEN are set!')
else:
    print('❌API KEY and TOKEN 404')

✅API KEY and TOKEN are set!


In [6]:
import requests
from pprint import pprint
url = 

header = {
    "accept": "application/json",
    "Authorization": f"Bearer {TMDB_API_TOKEN}"
}

response = requests.get(url, headers=headers)

pprint(response.json())

SyntaxError: invalid syntax (2885857037.py, line 3)